In [1]:
import os
os.chdir('/home/megatron/work')

import numpy as np
import pandas as pd
import megatron
from sklearn.preprocessing import StandardScaler

ImportError: cannot import name 'wrappers'

In [2]:
# from dataframe, lazy

df = pd.DataFrame({'a': np.arange(50), 'b': np.arange(50), 'c': np.arange(50)})
P = megatron.Pipeline()

col_nodes = megatron.adapters.from_dataframe(df, G)

T = megatron.transforms.Add()
out = T([col_nodes['a'], col_nodes['b'], col_nodes['c']])

result = P.run(out, df, cache_result=False)
print(result.shape)

(50,)


In [3]:
# from dataframe, eager

df = pd.DataFrame({'a': np.arange(50), 'b': np.arange(50), 'c': np.arange(50)})
P = megatron.Pipeline()

col_nodes = megatron.adapters.from_dataframe(df, P, eager=True)

T = megatron.transforms.Add()
result = T([col_nodes['a'], col_nodes['b'], col_nodes['c']])

print(result.output.shape)

(50,)


In [5]:
# lazy

P = megatron.Pipeline(cache_dir='megatron/cache')
X = megatron.Input(P, 'X', (10,))
Y = megatron.Input(P, 'Y', (10,))

T = megatron.wrappers.SklearnTransformation(StandardScaler())
X1 = T(X)
Y1 = T(Y)
X2 = megatron.transforms.TimeSeries(window_size=5)(X1)
Y2 = megatron.transforms.TimeSeries(window_size=5)(Y1)
Z = megatron.transforms.Add()([X2, Y2])

data = {'X': np.random.random((50, 10)),
        'Y': np.random.random((50, 10))}

result = P.run(Z, data, cache_result=True)
print(result.shape)

# test caching
result = P.run(Z, data, cache_result=True)
print(result.shape)

P.save('/home/megatron/work/test.p')

(45, 5, 10)
Loading node number 6 in path from cache
(45, 5, 10)


In [6]:
# eager

data = {'X': np.random.random((50, 10)),
        'Y': np.random.random((50, 10))}

P = megatron.Pipeline()
X = megatron.Input(P, 'X', (10,))(data['X'])
Y = megatron.Input(P, 'Y', (10,))(data['Y'])
X1 = megatron.transforms.TimeSeries(window_size=5)(X)
Y1 = megatron.transforms.TimeSeries(window_size=5)(Y)
Z = megatron.transforms.Add()([X1, Y1])

print(X.output.shape)
print(Z.output.shape)

(50, 10)
(45, 5, 10)


In [7]:
# reloading a graph

P2 = megatron.load_graph('/home/megatron/work/test.p')
P2.run('Add:0', data, cache_result=True).shape

AttributeError: 'str' object has no attribute 'input_nodes'

In [8]:
for node in P2.nodes:
    print(node.name)

X
Y
SklearnTransformation
SklearnTransformation
window(5)
window(5)
Add
